<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/minimal_circuit_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup
(No need to change anything)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-i48h2zyc
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-i48h2zyc
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 3f929b1d142b8f82bfbb8ae30e69bab7f76cadf3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9063, done.
remote: Counting objects: 100% (9063/9063), done.
remote: Compressing objects: 100% (3540/3540), done.
remote: Total 9063 (delta 5508), reused 8890 (delta 5425), pack-reused 0
Receiving objects: 100% (9063/9063), 155.49 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (5508/5508), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [11]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S5"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S4"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'S5')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [12]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'S5': str(i+4),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 11)
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [13]:
def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+2),
            'S5': str(i+3),
            'text': f"{i} {i+1} {i+2} {i+2}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 11)
dataset_2 = Dataset(prompts_list_2, model.tokenizer, S1_is_first=True)

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [14]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [15]:
def mean_ablate_by_lst(lst, model, print_output=True):
    CIRCUIT = {
        "number mover": lst,
        "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }

    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

# Ablate the model and compare with original

## Work backwards

https://www.notion.so/wlg1/Search-Methods-brainstorm-15a3020ab00b40adb79b0acf3622f5f4?pvs=4#dd6b43247d4945eda1d70ca4d4bae01d

In [35]:
# Start with full circuit
curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]
threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

for layer in range(11, -1, -1):  # go thru all heads in a layer first
    for head in range(12):
        # Copying the curr_circuit so we can iterate over one and modify the other
        copy_circuit = curr_circuit.copy()

        # Temporarily removing the current tuple from the copied circuit
        copy_circuit.remove((layer, head))

        new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

        # print((layer,head), new_score)
        # If the result is less than the threshold, remove the tuple from the original list
        if (100 - new_score) < threshold:
            curr_circuit.remove((layer, head))

            print("Removed:", (layer, head))
            print(new_score)
            print("\n")

Removed: (11, 0)
99.02855682373047


Removed: (11, 1)
99.0681381225586


Removed: (11, 2)
99.27287292480469


Removed: (11, 3)
99.61582946777344


Removed: (11, 4)
100.09127807617188


Removed: (11, 5)
100.09712982177734


Removed: (11, 6)
100.05851745605469


Removed: (11, 7)
99.9634017944336


Removed: (11, 8)
99.3410873413086


Removed: (11, 9)
99.123291015625


Removed: (11, 10)
98.09922790527344


Removed: (11, 11)
99.79859161376953


Removed: (10, 0)
99.7418212890625


Removed: (10, 1)
98.1563491821289


Removed: (10, 2)
100.61833953857422


Removed: (10, 3)
100.79714965820312


Removed: (10, 4)
100.48069763183594


Removed: (10, 5)
100.26615142822266


Removed: (10, 6)
100.29136657714844


Removed: (10, 8)
100.43941497802734


Removed: (10, 9)
100.6727294921875


Removed: (10, 10)
101.22868347167969


Removed: (10, 11)
100.94214630126953


Removed: (9, 0)
100.90746307373047


Removed: (9, 2)
100.99788665771484


Removed: (9, 3)
102.2463150024414


Removed: (9, 4)
101.65393829345

In [38]:
mean_ablate_by_lst(curr_circuit, model, print_output=True).item()

Average logit difference (circuit / full) %: 97.2790


97.27898406982422

In [37]:
curr_circuit

[(0, 1),
 (0, 3),
 (0, 5),
 (0, 7),
 (0, 9),
 (0, 10),
 (1, 0),
 (1, 4),
 (1, 5),
 (2, 2),
 (2, 8),
 (2, 9),
 (3, 0),
 (3, 2),
 (3, 3),
 (3, 7),
 (4, 4),
 (4, 7),
 (4, 10),
 (5, 1),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 8),
 (5, 9),
 (6, 1),
 (6, 4),
 (6, 6),
 (6, 10),
 (6, 11),
 (7, 6),
 (7, 10),
 (7, 11),
 (8, 0),
 (8, 5),
 (8, 6),
 (8, 8),
 (9, 1),
 (10, 7)]

Now try 10% threshold:

In [19]:
def find_circuit_backw(threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    # Start with full circuit
    curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(11, -1, -1):  # go thru all heads in a layer first
        for head in range(12):
            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, print_output=False).item()

            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("Removed:", (layer, head))
                print(new_score)
                print("\n")

    return curr_circuit

In [20]:
curr_circuit = find_circuit_backw(10)

Removed: (11, 0)
99.02855682373047


Removed: (11, 1)
99.0681381225586


Removed: (11, 2)
99.27287292480469


Removed: (11, 3)
99.61582946777344


Removed: (11, 4)
100.09127807617188


Removed: (11, 5)
100.09712982177734


Removed: (11, 6)
100.05851745605469


Removed: (11, 7)
99.9634017944336


Removed: (11, 8)
99.3410873413086


Removed: (11, 9)
99.123291015625


Removed: (11, 10)
98.09922790527344


Removed: (11, 11)
99.79859161376953


Removed: (10, 0)
99.7418212890625


Removed: (10, 1)
98.1563491821289


Removed: (10, 2)
100.61833953857422


Removed: (10, 3)
100.79714965820312


Removed: (10, 4)
100.48069763183594


Removed: (10, 5)
100.26615142822266


Removed: (10, 6)
100.29136657714844


Removed: (10, 8)
100.43941497802734


Removed: (10, 9)
100.6727294921875


Removed: (10, 10)
101.22868347167969


Removed: (10, 11)
100.94214630126953


Removed: (9, 0)
100.90746307373047


Removed: (9, 2)
100.99788665771484


Removed: (9, 3)
102.2463150024414


Removed: (9, 4)
101.65393829345

Try this method on greater-than task to see if recovers circuit similar to paper.

In [21]:
curr_circuit

[(0, 1),
 (0, 3),
 (0, 5),
 (0, 7),
 (0, 9),
 (0, 10),
 (1, 0),
 (1, 5),
 (2, 2),
 (2, 9),
 (3, 0),
 (3, 3),
 (3, 7),
 (4, 4),
 (4, 7),
 (4, 8),
 (4, 10),
 (5, 1),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 8),
 (5, 9),
 (5, 10),
 (6, 1),
 (6, 3),
 (6, 4),
 (6, 6),
 (6, 10),
 (7, 2),
 (7, 6),
 (7, 10),
 (7, 11),
 (8, 8),
 (9, 1),
 (10, 7)]

## etc fns

In [ ]:
# base_lst = [(0, 1), (0, 10), (3, 0), (4, 4), (5, 5), (6, 1), (6, 6), (7, 10), (7, 11), (8, 8), (8, 11), (9, 1), (9, 5), (10, 7)]

In [ ]:
# import json

# with open("scores.json", "w") as file:
#     json.dump(all_scores, file, default=lambda x: str(x))  # Convert tuples to strings for JSON serialization

In [ ]:
# from google.colab import files
# files.download("scores.json")  # or "scores.pkl" or "scores.json" depending on the file you saved

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>